In [141]:
import pandas as pd
import requests
import json
from dotenv import load_dotenv
import os
import time
import matplotlib.pyplot as plt
import numpy as np

In [142]:
os.listdir("../../data/static_files/")

['LargeCapCD.xlsx',
 'LargeCapComms.xlsx',
 'LargeCapCS.xlsx',
 'LargeCapEner.xlsx',
 'LargeCapFina.xlsx',
 'LargeCapHC.xlsx',
 'LargeCapInd.xlsx',
 'LargeCapIT.xlsx',
 'LargeCapMat.xlsx',
 'LargeCapRE.xlsx',
 'LargeCapUtil.xlsx',
 'nasdaq_nyse.xlsx']

In [143]:
load_dotenv("../../.env",override=True)
consumerKey = os.environ.get("CONSUMER_KEY")


In [144]:
pd.set_option("display.max_columns",500)
pd.set_option("display.max_rows",10)

In [145]:
def request_fundamentals(Tickers: list, consumerKey: str, how: str = 'DF') -> pd.DataFrame:
    """_summary_

    Args:
        Tickers (list)
        consumerKey (str)

    Returns:
        pd.DataFrame
    """
    symbolList = ",".join([i for i in Tickers])
    endpoint = f'''https://api.tdameritrade.com/v1/instruments?
        &symbol={symbolList}&projection=fundamental'''
    page = requests.get(url=endpoint, 
        params={'apikey' : consumerKey})
    content = json.loads(page.content)

    allDict = {}
    for k in list(content.keys()):
        allDict[k] = content[k]['fundamental']
    fundamentals = pd.DataFrame(allDict).T
    
    if how=='DF':
        return fundamentals
    else:
        return allDict

def request_quotes(consumerKey: str, Tickers: list) -> pd.DataFrame:
    """_summary_

    Args:
        consumerKey (str): _description_
        Tickers (list): _description_

    Returns:
        pd.DataFrame: _description_
    """
    
    symbolList = ",".join([i for i in Tickers])
    endpoint = f"https://api.tdameritrade.com/v1/marketdata/quotes?symbol={symbolList}"
    page = requests.get(url=endpoint, 
                params={'apikey' : consumerKey})
    content = json.loads(page.content)
    content = pd.DataFrame(content).T
    return content

In [146]:
suffixes = ['CD','Comms','CS','Ener','Fina','HC','Ind','IT','Mat','RE','Util']

In [147]:
Industry = suffixes[7]
symbols = pd.read_excel(f"../../data/static_files/LargeCap{Industry}.xlsx",header=1)
symbols['Symbol'].fillna("-",inplace=True)
Tickers = list(symbols[(~symbols['Symbol'].astype(str).str.contains('-'))]['Symbol'])


c:\Users\jliv\Desktop\research\python_research\.venv\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [148]:

FDF = request_fundamentals(Tickers=Tickers,consumerKey=consumerKey)
Quotes = request_quotes(Tickers=Tickers,consumerKey=consumerKey)

In [150]:
dat = Quotes[['symbol','description','lastPrice',
    'volatility','peRatio','divAmount','divYield',
    ]].merge(FDF[['symbol']+[
        i for i in FDF.columns if i not in Quotes.columns
    ]],on='symbol',how='inner')

dat=pd.DataFrame(np.where(dat==0,np.nan,dat),columns = dat.columns)

In [151]:
def MinMaxScaler(x: pd.Series) -> pd.Series:
    """_summary_

    Args:
        x (pd.Series): _description_

    Returns:
        pd.Series: _description_
    """
    return (x-np.nanmin(x))/(np.nanmax(x)-np.nanmin(x))

def MaxMinScaler(x: pd.Series) -> pd.Series:
    """_summary_

    Args:
        x (pd.Series): _description_

    Returns:
        pd.Series: _description_
    """
    return 1-MinMaxScaler(x)

In [152]:
rankdat = dat.copy()

In [153]:
metricsFamilies = {
    'debt': {'columns': ['totalDebtToCapital','ltDebtToEquity','totalDebtToEquity'],
            'ascending':True},
    'ratio': {'columns': ['quickRatio','currentRatio','interestCoverage'],
            'ascending':False},
    'change': {'columns': ['epsChangePercentTTM',
       'epsChangeYear', 'epsChange', 'revChangeYear', 'revChangeTTM',
       'revChangeIn'],
            'ascending':False},
    'profit': {'columns': ['epsTTM','grossMarginTTM', 'grossMarginMRQ', 'netProfitMarginTTM',
       'netProfitMarginMRQ', 'operatingMarginTTM', 'operatingMarginMRQ'],
            'ascending':False},
    'value': {'columns': ['peRatio','pegRatio', 'pbRatio', 'prRatio',
       'pcfRatio'],
            'ascending':True},
    'return': {'columns':  ['returnOnEquity', 'returnOnAssets', 'returnOnInvestment'],
            'ascending':False},
}

In [154]:
def convert_to_ranking(rankdat: pd.DataFrame, 
    metricsFamilies: dict[str,dict]) -> pd.DataFrame:
    """_summary_

    Args:
        rankdat (pd.DataFrame): _description_
        metricsFamilies (dict[str,dict]): _description_

    Returns:
        pd.DataFrame: _description_
    """
    families = list(metricsFamilies.keys())
    
    for f in families:
        columns = metricsFamilies[f]['columns']
        ascending= metricsFamilies[f]['ascending']
        for v in columns:
            rankdat[v] = rankdat[v].rank(ascending=ascending)

        rankdat[f'{f}Known']=np.sum(np.where(rankdat[columns].isna(),0,1),axis=1)

        rankdat[columns]=rankdat[columns]/np.nanmax(rankdat[columns],axis=0)

    return rankdat


In [155]:
rankdat = convert_to_ranking(rankdat,metricsFamilies)

C:\Users\jliv\AppData\Local\Temp\ipykernel_14784\933486261.py:22: RuntimeWarning: All-NaN axis encountered
  rankdat[columns]=rankdat[columns]/np.nanmax(rankdat[columns],axis=0)


In [157]:
rankedCols = []
for k in metricsFamilies.keys():
    rankedCols=rankedCols+metricsFamilies[k]['columns']
    
knownCols = [i for i in rankdat.columns if 'Known' in i]
rankdat['metricsInformed'] = np.sum(rankdat[knownCols],axis=1)


In [158]:
rankdat['meanRank']=np.sum(rankdat[rankedCols],axis=1)/rankdat['metricsInformed']

In [159]:
out=rankdat[['symbol','description','meanRank','metricsInformed']].merge(
    dat[['symbol']+rankedCols],
    on=['symbol'],
    how='left'
)

In [172]:
np.round(out.sort_values(by='meanRank').head(10),2)

,symbol,description,meanRank,metricsInformed,totalDebtToCapital,ltDebtToEquity,totalDebtToEquity,quickRatio,currentRatio,interestCoverage,epsChangePercentTTM,epsChangeYear,epsChange,revChangeYear,revChangeTTM,revChangeIn,epsTTM,grossMarginTTM,grossMarginMRQ,netProfitMarginTTM,netProfitMarginMRQ,operatingMarginTTM,operatingMarginMRQ,peRatio,pegRatio,pbRatio,prRatio,pcfRatio,returnOnEquity,returnOnAssets,returnOnInvestment
115,PATH,"UiPath, Inc. Class A Common Stock",0.23,6,NaN,NaN,NaN,NaN,4.58293,NaN,NaN,NaN,NaN,NaN,35.40498,NaN,NaN,82.6219,81.78572,NaN,NaN,NaN,NaN,NaN,NaN,3.49061,6.4721,NaN,NaN,NaN,NaN
146,TER,"Teradyne, Inc. - Common Stock",0.25,20,3.13642,2.81894,3.23798,2.47661,2.84735,NaN,NaN,NaN,NaN,NaN,NaN,11.30519,4.74696,60.01053,60.22948,24.49157,23.52462,28.82248,29.49572,15.3319,NaN,5.30732,3.5302,12.68109,34.92339,22.44449,29.45617
150,TWLO,Twilio Inc. Class A Common Stock,0.26,10,8.58174,9.27462,9.38733,NaN,6.41803,NaN,NaN,NaN,NaN,NaN,50.85012,7.76718,NaN,47.96285,47.20275,NaN,NaN,NaN,NaN,NaN,NaN,1.27413,4.06181,NaN,NaN,NaN,NaN
69,HCP,"HashiCorp, Inc. - Class A Common Stock",0.28,7,NaN,NaN,NaN,NaN,4.71571,NaN,NaN,NaN,NaN,NaN,51.88103,12.85073,NaN,79.9814,81.08253,NaN,NaN,NaN,NaN,NaN,NaN,4.63139,14.48422,NaN,NaN,NaN,NaN
149,TSM,Taiwan Semiconductor Manufacturing Company Ltd.,0.28,24,26.70323,31.23432,36.63757,1.99757,2.25482,NaN,37.56191,76.42164,NaN,NaN,29.29828,8.76951,4.5822,54.96044,59.06083,40.62169,44.40407,44.70883,49.07182,13.99,0.370389,4.24822,5.37493,8.37775,33.96735,20.51166,25.73269
155,UMC,United Microelectronics Corporation (NEW) Comm...,0.29,24,20.44695,18.06314,25.72732,2.40852,2.68129,NaN,73.19796,87.62883,NaN,NaN,26.25198,7.31516,0.82099,37.96148,43.36661,28.46849,31.63786,28.95249,35.21475,6.8,0.103837,1.65464,2.12425,4.3554,23.62713,14.88621,18.51822
99,MU,"Micron Technology, Inc. - Common Stock",0.29,22,12.15567,13.63135,13.83774,2.00531,2.88911,NaN,50.64456,NaN,NaN,NaN,11.01967,NaN,7.7337,45.18499,39.47012,28.23005,22.535,31.27316,22.89628,6.7949,0.143105,1.22866,1.98114,3.85694,18.51449,13.87814,15.62126
139,STM,STMicroelectronics N.V. Common Stock,0.31,24,19.77082,23.48134,24.80063,1.65679,2.27829,NaN,76.0977,106.1272,NaN,NaN,18.70014,8.20643,3.03323,45.36644,47.40683,20.1047,22.6479,23.84692,26.16628,10.3,0.14431,2.97653,2.10106,7.52676,30.1616,17.33402,22.31032
151,TXN,Texas Instruments Incorporated - Common Stock,0.31,25,33.95201,47.86404,51.40505,4.23594,5.09492,56.91837,27.53735,19.44764,NaN,NaN,16.88343,6.25892,9.14467,69.25786,69.55104,43.77807,43.95625,51.52103,52.24482,16.5681,0.634263,10.37069,7.44882,15.36668,67.65696,37.84583,42.28769
118,PCOR,"Procore Technologies, Inc. Common Stock",0.32,10,3.89544,4.05333,4.05333,NaN,1.70697,NaN,NaN,52.90595,NaN,NaN,NaN,7.95469,NaN,79.90748,78.66787,NaN,NaN,NaN,NaN,NaN,NaN,6.25204,11.99129,NaN,NaN,NaN,NaN


In [177]:
for i in out.columns:
    try:
        out[i]=out[i].astype(float)
    except:
        pass

In [179]:
np.round(out,2)

,symbol,description,meanRank,metricsInformed,totalDebtToCapital,ltDebtToEquity,totalDebtToEquity,quickRatio,currentRatio,interestCoverage,epsChangePercentTTM,epsChangeYear,epsChange,revChangeYear,revChangeTTM,revChangeIn,epsTTM,grossMarginTTM,grossMarginMRQ,netProfitMarginTTM,netProfitMarginMRQ,operatingMarginTTM,operatingMarginMRQ,peRatio,pegRatio,pbRatio,prRatio,pcfRatio,returnOnEquity,returnOnAssets,returnOnInvestment
0,AAPL,Apple Inc. - Common Stock,0.53,22.0,67.32,162.98,205.98,0.82,0.86,NaN,18.52,NaN,NaN,NaN,11.63,NaN,6.05,43.31,43.26,25.71,23.44,30.53,27.82,23.75,1.31,40.79,6.11,21.35,162.82,29.91,46.50
1,ACN,Accenture plc Class A Ordinary Shares (Ireland),0.47,22.0,0.24,0.21,0.25,NaN,1.23,NaN,17.02,18.42,NaN,NaN,21.89,NaN,10.71,31.99,32.07,11.35,10.97,15.05,14.71,24.08,1.48,7.70,2.91,20.95,33.04,15.46,24.96
2,ADBE,Adobe Inc. - Common Stock,0.35,21.0,22.31,25.23,28.71,NaN,1.14,43.65,NaN,NaN,NaN,NaN,13.86,1.07,10.14,87.76,87.68,28.00,25.63,35.45,33.48,29.67,NaN,9.95,8.29,25.17,33.44,18.20,24.52
3,ADI,"Analog Devices, Inc. - Common Stock",0.46,23.0,14.58,17.07,17.07,1.43,1.92,22.74,NaN,6.30,NaN,NaN,70.73,4.64,3.61,58.63,65.70,17.00,24.08,18.55,28.72,39.08,NaN,2.06,6.79,18.43,7.72,5.25,5.66
4,ADP,"Automatic Data Processing, Inc. - Common Stock",0.58,23.0,48.09,92.61,92.65,NaN,0.99,59.20,15.38,18.25,NaN,NaN,9.95,NaN,7.00,42.81,42.35,17.87,15.15,22.91,19.60,33.56,2.16,30.02,5.86,27.90,66.30,5.27,31.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,WU,Western Union Company (The) Common Stock,0.51,20.0,85.74,543.23,601.23,NaN,NaN,11.52,14.10,NaN,NaN,NaN,NaN,NaN,2.23,43.61,42.63,18.27,17.04,26.48,23.19,6.40,0.43,11.74,1.08,4.87,241.98,10.01,12.16
165,XM,Qualtrics International Inc. - Class A Common ...,0.46,8.0,NaN,NaN,NaN,NaN,1.34,NaN,NaN,6.54,NaN,NaN,43.08,6.17,NaN,71.68,70.69,NaN,NaN,NaN,NaN,NaN,NaN,3.22,4.84,NaN,NaN,NaN,NaN
166,ZBRA,Zebra Technologies Corporation - Class A Commo...,0.61,20.0,46.23,80.26,85.99,0.56,0.85,40.50,NaN,NaN,NaN,NaN,12.37,2.51,9.20,45.25,45.91,8.56,NaN,9.08,NaN,27.43,NaN,5.44,2.35,19.49,19.48,7.66,10.67
167,ZM,"Zoom Video Communications, Inc. - Class A Comm...",0.50,18.0,NaN,NaN,NaN,NaN,3.44,3.51,NaN,NaN,NaN,NaN,18.09,2.39,3.23,75.24,75.11,23.06,4.16,19.82,11.07,24.08,NaN,4.03,5.61,22.95,18.60,13.61,18.00
